In [6]:
from TTS.api import TTS
import speech_recognition as sr
import os
from playsound import playsound
import time
import openai
import weaviate
import json
import requests

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    

# List available 🐸TTS models and choose the first one
model_name = 'tts_models/en/ljspeech/tacotron2-DDC'
# considering switchign to pytts3x

# Init TTS
tts = TTS(model_name)

# Create a recognizer object
r = sr.Recognizer()

# Prompt the user to speak
print("Please speak now...")


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

In [7]:
playsound(os.path.join(os.getcwd(), 'please_speak.wav'))


In [8]:
with sr.Microphone() as source:
    audio = r.listen(source)
print("Processing...")


Processing...


In [9]:
start_time = time.time()
playsound(os.path.join(os.getcwd(), 'ipt_recv.wav'))

In [10]:
try:
    # Convert speech to text
    text = r.recognize_google(audio)
    # text = "got that promotion at work! 🎉🙌🏼 But at what cost 😔💔"
    #text = "What does Assorted really mean? I see it in a box of chocolates being represented as a varied collection, but other times I see it as many items of the same quality and such?"

except sr.UnknownValueError:
    playsound(os.path.join(os.getcwd(), 'unable_to_understand.wav')) 

print("Your text input is ", text)

Your text input is  how is the weather in Chennai


In [11]:
# alright crazy idea
# instead of some bigass RAKE up my codebase or doing logn NLTK stuff
# what if i just .... 
# use openAI for keyword extraction?????
# god i may be a genius (re: Sarcasm)
# text = "Who are the legendary Vampire Counts of the Von Carstein bloodline?"
input_prompt = """
What are the keywords or phrases from the sentence :
"""+text+"\nOutput:"
print("The input prompt is")
print(input_prompt)
# input_prompt = """
# Classify tweet as positive, negative, or neutral
# (Take into account hidden meaning of emojis, and meme culture context): 
# """ + text + "\nOutput:"
# print(input_prompt)


The input prompt is

What are the keywords or phrases from the sentence :
how is the weather in Chennai
Output:


In [12]:
api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key
# shifting model to 3.5 turbo, saves SOO many tokens
# also chat completion instead of simple completion
# already trained, just need to piepline
GPT_MODEL = "gpt-3.5-turbo-0613"

In [13]:
response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
    {"role" : "system", "content" : "You are to find upto 3 keywords or keyphrases from a given sentence"},
    {"role" : "system", "content" : "Restrict yourself to ONLY provide the answer, and nothing else"},
    {"role" : "system", "content" : "Provide the response as a list of words which are comma separated"},
    {"role" : "user", "content" : input_prompt},
  ]
)

keyphrases = response["choices"][0]["message"]["content"]
print(keyphrases)

weather, Chennai


In [14]:
# dummy for db/context access
# split keywords across ,
# strip spaces
# search context db for keyword
# add context (if found) to the input


In [15]:
weaviate_client_url = "http://localhost:8080"

# docker-compose up -d <- to create
# docker-compose start <- to start
# docker-compose down <- to remove and exit
# docker-compose stop <- to stop

client = weaviate.Client(
    url=weaviate_client_url,  # Replace with your endpoint
    additional_headers={
        "X-OpenAI-Api-Key": api_key  # Or "X-Cohere-Api-Key" or "X-HuggingFace-Api-Key"
    }
)

# some_objects = client.data_object.get()
# # print(json.dumps(some_objects))
# print("Article title  : " + some_objects['objects'][0]['properties']['title'])
# print("Article sample : " + some_objects['objects'][0]['properties']['content'][0:80])

In [16]:
contents = list()
title = list()
# keywords = ["Hitler", "Xianxia", "wallet"]
context = "Context Below -> "
keywords = keyphrases.split(",")
keywords = [word.strip() for word in keywords]

print(keywords)


nearText = {"concepts" : text}
nearResponse = (
    client.query
    .get("Article", ["title", "content"])
    .with_near_text(nearText)
    .with_limit(1)
    .do()
)

wiki_content = nearResponse['data']['Get']['Article'][0]['content']
lines = wiki_content.split('.')

wiki_title = nearResponse['data']['Get']['Article'][0]['title']
context = context + "\n" + wiki_title + "\n"

print("Title :", wiki_title, "Content :")
for line in lines[0:2]:
    if line == "":
        break
    context = context + line + "\n"
    print(line)
print("\n")

for keyword in keywords:
    print("For keyword :", keyword)
    response = (
        client.query
        .get("Article", ["title", "content"])
        .with_hybrid(keyword, alpha=0.5)  # default 0.75
        .with_limit(1)
        .do()
    )
    
    if response == nearResponse:
        continue

    wiki_content = response['data']['Get']['Article'][0]['content']
    lines = wiki_content.split('.')
    
    wiki_title = response['data']['Get']['Article'][0]['title']
    context = context + "\n" + wiki_title + "\n"
    
    print("Title :", wiki_title, "\nContent :")
    for line in lines[0:2]:
        # print("len of line is ", len(line))
        if line[0] == "\n":
            # print("entered inside")
            break
        context = context + line + "\n"
        print(line)
    print("\n")
context = context + "\n<- Context Above"
print(context)

['weather', 'Chennai']
Title : Chennai Content :
Chennai (formerly known as Madras) is the capital city of the Indian state of Tamil Nadu
 It has a population of about 7 million people


For keyword : weather
Title : National Weather Service 
Content :
The National Weather Service (also known as NWS) is one of the six scientific agencies that make up the National Oceanic and Atmospheric Administration (NOAA) of the United States government
 Its job is to provide "weather, hydrologic, and climate forecasts and warnings for the United States, its territories, adjacent waters and ocean areas, for the protection of life and property and the enhancement of the national economy


For keyword : Chennai
Context Below -> 
Chennai
Chennai (formerly known as Madras) is the capital city of the Indian state of Tamil Nadu
 It has a population of about 7 million people

National Weather Service
The National Weather Service (also known as NWS) is one of the six scientific agencies that make up the Nat

In [17]:
# context = """
# Vampire
# Vampires are monsters in legends and stories
#  The first vampire stories were told in Eastern Europe, but much of how modern people see vampires was created by Bram Stoker in the famous novel, Dracula

# Vampire bat
# Vampire bats is a subfamily of bats
#  Unlike other bats, vampire bats feed on blood

# Warhammer 40,000
# Warhammer 40,000 , also called 40k, is a table-top game and fictional setting made by Games Workshop, in which model armies fight against each other on miniature terrain

# Bismarck
# Bismarck is a German family name
#  Bismark is an incorrect spelling
# """

In [18]:
# text = "sample text, Hi Who Are You?"
# context = "sample context, Himeko from Honkai Impact"

# do i need a 'personality' ??
# or is the default fine?
# im kinda not making a neuro-mini anymore
# just doing my own thing
# remind myself -> rethink personality inputs

personality = """
You are a personal assistant. Your job is to answer the user queries.
You will be provided some additional info, and have to decide if info is relevant to the query.
The user is unaware of the info provided, and do not refer the existence of this info in your answer"""
user_prompt = text

print("\nThe input prompt is")
print(user_prompt)
print("\nGPT Instructions:")
print(personality)
print("\nContext is given as:")
print(context)


The input prompt is
how is the weather in Chennai

GPT Instructions:

You are a personal assistant. Your job is to answer the user queries.
You will be provided some additional info, and have to decide if info is relevant to the query.
The user is unaware of the info provided, and do not refer the existence of this info in your answer

Context is given as:
Context Below -> 
Chennai
Chennai (formerly known as Madras) is the capital city of the Indian state of Tamil Nadu
 It has a population of about 7 million people

National Weather Service
The National Weather Service (also known as NWS) is one of the six scientific agencies that make up the National Oceanic and Atmospheric Administration (NOAA) of the United States government
 Its job is to provide "weather, hydrologic, and climate forecasts and warnings for the United States, its territories, adjacent waters and ocean areas, for the protection of life and property and the enhancement of the national economy

<- Context Above


In [31]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, eg. San Francisco, or Chennai",
                },
            },
            "required": ["location"],
        },
    }]

def get_current_weather(location):
    # print("test")
    # print(location)
    # print(format)
    api_key = os.environ.get('weather_api_key')
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + location
    response = requests.get(complete_url)
    x = response.json()
    if x["cod"] != "404":
        y = x["main"]
        current_temperature = y["temp"]
        z = x['weather']
        weather_description = z[0]["description"]
        return ("temp is "+str(round(current_temperature-273.15, 2))+" and weather is "+str(weather_description))

function_exec = {
    'get_current_weather' : get_current_weather
}

In [32]:
query_response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
    {"role" : "system", "content" : personality+context},
    {"role" : "user", "content" : user_prompt}
  ],
  functions=functions
)
# choices = query_response['choices'][0]
# finish = choices['finish_reason']
# if finish == 'function_call':
#   function_exec[choices['message']['function_call']]()

# answer = query_response["choices"][0]["message"]["content"]

In [33]:
# print(query_response['choices'][0]['finish_reason'])
# finish = query_response['choices'][0]['finish_reason']
# if finish == 'function_call':
#   print("test")
print(query_response['choices'][0]['message']['function_call']['name'])
print(query_response['choices'][0]['message']['function_call']['arguments'])

choices = query_response['choices'][0]
finish = choices['finish_reason']
if finish == 'function_call':
  func_params = choices['message']['function_call']
  func_name = func_params['name']
  func_inputs = json.loads(func_params['arguments'])
  print("func inputs full", func_inputs)
  print("location", func_inputs.get('location'))
  output = function_exec[choices['message']['function_call']['name']](func_inputs.get("location"))
  print(output)


get_current_weather
{
  "location": "Chennai"
}
func inputs full {'location': 'Chennai'}
location Chennai
temp is 34.98 and weather is few clouds


In [ ]:
for sentences in answer.split(". "):
    print(sentences)

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
# testing the pytts3x for synthing
import pyttsx3
tester_engine = pyttsx3.init()
tester_engine.setProperty('rate', 175)
tester_engine.setProperty('voice', tester_engine.getProperty('voices')[1].id)
tester_engine.say(answer)
tester_engine.runAndWait()
tester_engine.stop()

In [ ]:
# Run TTS on the text
# tts.tts_to_file(reply, file_path='output.wav')
# second idea, can i maybe stream RVC through this?
# or SOVITS

In [ ]:
end_time = time.time()
# import playsound
# import os
# playsound(os.path.join(os.getcwd(), 'output.wav'))

print("time taken : ", -start_time + end_time)

time taken :  28.53394913673401
